In [1]:
import cv2
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
#from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image

In [2]:
# Define the directories for the 3 different image types
img_dir_1 = "C:/Users/saitu/Downloads/AIOralCancerProject/AIOralCancerProject/amalgamTattoo"
img_dir_2 = "C:/Users/saitu/Downloads/AIOralCancerProject/AIOralCancerProject/aphthousUlcer"
img_dir_3 = "C:/Users/saitu/Downloads/AIOralCancerProject/AIOralCancerProject/candidiasis"
# Define the class labels
class_labels = ["amalgamTattoo", "aphthousUlcer", "candidiasis" ]

In [20]:
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing import image

# Define the base directory
base_dir = "C:/Users/saitu/Downloads/AIOralCancerProject/AIOralCancerProject"

# Create an empty list to store the images and labels
X = []
y = []

# Get the class labels from the directory names
class_labels = os.listdir(base_dir)

# Iterate through each directory and store the images and labels
for i, label in enumerate(class_labels):
    img_dir = os.path.join(base_dir, label)
    for file in os.listdir(img_dir):
        # Check if file is an image
        if file.endswith(".jpg") or file.endswith(".png"):
            # Load the image
            
            img = Image.open(os.path.join(img_dir, file))
            img = img.resize((224, 224))
            img = np.array(img)
            img = img / 255.0

            # Append the image to the list of images
            X.append(img)
            # Append the corresponding label to the list of labels
            y.append(i)

In [6]:
# Create an empty list to store the images and labels
X = []
y = []

# Iterate through each directory and store the images and labels
for i, img_dir in enumerate([img_dir_1, img_dir_2, img_dir_3]):
    for file in os.listdir(img_dir):
        # Check if file is an image
        if file.endswith(".jpg") or file.endswith(".png"):
            # Load the image
            img = Image.open(os.path.join(img_dir, file))
            img = img.resize((224, 224))
            img = np.array(img)
            img = img / 255.0
            #img = Image.img_to_array(img)
            #img = preprocess_input(img)

            # Append the image to the list of images
            X.append(img)
            # Append the corresponding label to the list of labels
            y.append(i)

In [22]:
# Shuffle the data
X, y = shuffle(X, y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.reshape(-1,224,224,3)
X_test = X_test.reshape(-1,224,224,3)

In [15]:
# Define the model architecture
input_shape = (224, 224, 3)
input_tensor = Input(shape=input_shape)

x = ZeroPadding2D((3, 3))(input_tensor)
x = Conv2D(32, (3, 3), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, name='bn1')(x)
x = Activation('relu')(x)
x = MaxPooling2D

In [24]:
# Define the model architecture
input_shape = (224, 224, 3)
input_tensor = Input(shape=input_shape)

x = ZeroPadding2D((3, 3))(input_tensor)
x = Conv2D(32, (3, 3), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, name='bn1')(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), name='max_pool1')(x)

# Define the output layer
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(len(class_labels), activation='softmax')(x)

# Create the model
model = Model(input_tensor, x)


In [25]:
from keras.utils import to_categorical

# One-hot encode the labels
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
checkpointer = ModelCheckpoint(filepath='model.h5', save_best_only=True)

# Train the model
history = model.fit(X_train, y_train_encoded, epochs=100, batch_size=32, validation_data=(X_test, y_test_encoded), callbacks=[early_stopping, checkpointer])


Epoch 1/100
70/70 [==============================] - 51s 601ms/step - loss: 6.0732 - accuracy: 0.1232 - val_loss: 2.7524 - val_accuracy: 0.1691
Epoch 2/100
70/70 [==============================] - 25s 354ms/step - loss: 2.7504 - accuracy: 0.1448 - val_loss: 2.7269 - val_accuracy: 0.1691
Epoch 3/100
70/70 [==============================] - 25s 353ms/step - loss: 2.7220 - accuracy: 0.1457 - val_loss: 2.7032 - val_accuracy: 0.1691
Epoch 4/100
70/70 [==============================] - 32s 467ms/step - loss: 2.7027 - accuracy: 0.1457 - val_loss: 2.6823 - val_accuracy: 0.1691
Epoch 5/100
70/70 [==============================] - 23s 323ms/step - loss: 2.6857 - accuracy: 0.1457 - val_loss: 2.6630 - val_accuracy: 0.1691
Epoch 6/100
70/70 [==============================] - 25s 364ms/step - loss: 2.6707 - accuracy: 0.1457 - val_loss: 2.6460 - val_accuracy: 0.1691
Epoch 7/100
70/70 [==============================] - 24s 347ms/step - loss: 2.6574 - accuracy: 0.1457 - val_loss: 2.6306 - val_accuracy:

70/70 [==============================] - 29s 417ms/step - loss: 2.5721 - accuracy: 0.1457 - val_loss: 2.5182 - val_accuracy: 0.1691
Epoch 58/100
70/70 [==============================] - 26s 375ms/step - loss: 2.5721 - accuracy: 0.1457 - val_loss: 2.5182 - val_accuracy: 0.1691
Epoch 59/100
70/70 [==============================] - 28s 400ms/step - loss: 2.5721 - accuracy: 0.1457 - val_loss: 2.5184 - val_accuracy: 0.1691
Epoch 60/100
70/70 [==============================] - 28s 401ms/step - loss: 2.5721 - accuracy: 0.1457 - val_loss: 2.5181 - val_accuracy: 0.1691
Epoch 61/100
70/70 [==============================] - 27s 391ms/step - loss: 2.5721 - accuracy: 0.1457 - val_loss: 2.5180 - val_accuracy: 0.1691
Epoch 62/100
70/70 [==============================] - 29s 421ms/step - loss: 2.5721 - accuracy: 0.1457 - val_loss: 2.5182 - val_accuracy: 0.1691


In [26]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print('Test accuracy:', test_acc)

18/18 [==============================] - 2s 107ms/step - loss: 2.5182 - accuracy: 0.1691
Test accuracy: 0.1690647453069687
